# Imports & Load Data
作業に必要なライブラリをインポートして、 以下のデータを読み込みます。

* stock_price : 株価情報
* stock_list : 銘柄情報
* stock_fin : 財務諸表
* stock_labels : 目的変数

In [1]:
import os
import pickle
import sys
import warnings
from glob import glob

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import  xgboost as xgb
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm.auto import tqdm


# 表示用の設定を変更します
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120

In [2]:
# python 3.7.3であることを確認します
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [3]:
# データセット保存先ディレクトリ（""の中身はご自身の環境に合わせて定義してください。）
dataset_dir="/path/to"

In [4]:
inputs = {"stock_price": f"{dataset_dir}/stock_price.csv.gz"}

In [5]:
df = pd.read_csv(inputs["stock_price"])
df.head(1)

,Local Code,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose,EndOfDayQuote PreviousCloseDate,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote VWAP
0,1301,2016/01/04,2800.0,2820.0,2740.0,2750.0,2750.0,32000.0,0.1,2770.0,2015/12/30,2770.0,2015/12/30,-20.0,-0.722,2778.25


In [6]:
df.loc[:, "datetime"] = pd.to_datetime(df.loc[:, "EndOfDayQuote Date"])
df.set_index("datetime", inplace=True)
df.head(1)

,Local Code,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose,EndOfDayQuote PreviousCloseDate,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote VWAP
datetime,,,,,,,,,,,,,,,,
2016-01-04,1301,2016/01/04,2800.0,2820.0,2740.0,2750.0,2750.0,32000.0,0.1,2770.0,2015/12/30,2770.0,2015/12/30,-20.0,-0.722,2778.25


In [7]:
feats = (
    df[["EndOfDayQuote ExchangeOfficialClose", "Local Code"]]
    .groupby("Local Code")
    .pct_change()
    .rolling(20)
    .std()
    .values
)
feats[:21]


array([[       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [       nan],
       [0.01590696]])

In [8]:
df.loc[:, "code"] = df.index.strftime("%Y-%m-%d-") + df.loc[
    :, "Local Code"
].astype(str)
df.head(1).T


datetime,2016-01-04 00:00:00
Local Code,1301
EndOfDayQuote Date,2016/01/04
EndOfDayQuote Open,2800
EndOfDayQuote High,2820
EndOfDayQuote Low,2740
EndOfDayQuote Close,2750
EndOfDayQuote ExchangeOfficialClose,2750
EndOfDayQuote Volume,32000
EndOfDayQuote CumulativeAdjustmentFactor,0.1
EndOfDayQuote PreviousClose,2770


In [9]:
df.loc[:, "label_high_20"] = feats
df.loc[:, "label_low_20"] = feats

In [10]:
# 出力対象列を設定
output_columns = ["code", "label_high_20", "label_low_20"]

In [11]:
import io
out = io.StringIO()
df[output_columns].loc["2020-01-01":].dropna().to_csv(out, header=False, index=False)

print("\n".join(out.getvalue().split("\n")[:10]))

2020-01-06-1301,0.003585883791862692,0.003585883791862692
2020-01-07-1301,0.004547330404501981,0.004547330404501981
2020-01-08-1301,0.00453195173316532,0.00453195173316532
2020-01-09-1301,0.004999748259166511,0.004999748259166511
2020-01-10-1301,0.004999647093872243,0.004999647093872243
2020-01-14-1301,0.005258378653780425,0.005258378653780425
2020-01-15-1301,0.0052927619941197995,0.0052927619941197995
2020-01-16-1301,0.005278015255069897,0.005278015255069897
2020-01-17-1301,0.005145885202200792,0.005145885202200792
2020-01-20-1301,0.0051441150911965775,0.0051441150911965775


In [18]:
import io

import pandas as pd

class ScoringService(object):
    @classmethod
    def get_model(cls, model_path="../model"):
        return True

    @classmethod
    def predict(cls, inputs):
        df = pd.read_csv(inputs["stock_price"])
        df.loc[:, "datetime"] = pd.to_datetime(df.loc[:, "EndOfDayQuote Date"])
        df.set_index("datetime", inplace=True)
        feats = (
            df[["EndOfDayQuote ExchangeOfficialClose", "Local Code"]]
            .groupby("Local Code")
            .pct_change()
            .rolling(20)
            .std()
            .values
        )
        df.loc[:, "code"] = df.index.strftime("%Y-%m-%d-") + df.loc[
        :, "Local Code"
        ].astype(str)
        df.loc[:, "label_high_20"] = feats
        df.loc[:, "label_low_20"] = feats
        output_columns = ["code", "label_high_20", "label_low_20"]
        out = io.StringIO()
        df[output_columns].loc["2020-01-01":].dropna().to_csv(out, header=False, index=False)

        return out.getvalue()

In [19]:
assert ScoringService.get_model()

In [20]:
actual = ScoringService.predict(inputs)
assert out.getvalue() == actual
print("\n".join(actual.split("\n")[:10]))

2020-01-06-1301,0.003585883791862692,0.003585883791862692
2020-01-07-1301,0.004547330404501981,0.004547330404501981
2020-01-08-1301,0.00453195173316532,0.00453195173316532
2020-01-09-1301,0.004999748259166511,0.004999748259166511
2020-01-10-1301,0.004999647093872243,0.004999647093872243
2020-01-14-1301,0.005258378653780425,0.005258378653780425
2020-01-15-1301,0.0052927619941197995,0.0052927619941197995
2020-01-16-1301,0.005278015255069897,0.005278015255069897
2020-01-17-1301,0.005145885202200792,0.005145885202200792
2020-01-20-1301,0.0051441150911965775,0.0051441150911965775


In [23]:
print("\n".join(actual.split("\n")[:500]))

2020-01-06-1301,0.003585883791862692,0.003585883791862692
2020-01-07-1301,0.004547330404501981,0.004547330404501981
2020-01-08-1301,0.00453195173316532,0.00453195173316532
2020-01-09-1301,0.004999748259166511,0.004999748259166511
2020-01-10-1301,0.004999647093872243,0.004999647093872243
2020-01-14-1301,0.005258378653780425,0.005258378653780425
2020-01-15-1301,0.0052927619941197995,0.0052927619941197995
2020-01-16-1301,0.005278015255069897,0.005278015255069897
2020-01-17-1301,0.005145885202200792,0.005145885202200792
2020-01-20-1301,0.0051441150911965775,0.0051441150911965775
2020-01-21-1301,0.005096458711001918,0.005096458711001918
2020-01-22-1301,0.005149269541044462,0.005149269541044462
2020-01-23-1301,0.0054601091075363786,0.0054601091075363786
2020-01-24-1301,0.005903761412883266,0.005903761412883266
2020-01-27-1301,0.006033626662618963,0.006033626662618963
2020-01-28-1301,0.006051684650732913,0.006051684650732913
2020-01-29-1301,0.005996201097735637,0.005996201097735637
2020-01-30